# PQF: Gravitational-Wave–Like Drift (Derived, Reviewer-Ready)

**Principle — no ad-hoc terms.**  
This notebook simulates envelope drift of a localized $\phi$ wave due to **inhomogeneous dispersion** derived from the LPQF Lagrangian:
\[\mathcal{L}=\tfrac12(\partial_t \phi)^2-\tfrac{c_\phi^2}{2}(\partial_x\phi)^2 - V(\phi) \; -\; \tfrac{\alpha}{2}\,\phi^2(\partial_x\theta)^2\]
Coupled equations of motion (Euler–Lagrange):
\[\phi_{tt}-c_\phi^2\phi_{xx}+V'(\phi)+\alpha\,\phi\,\theta_x^2=0,\qquad
\theta_{tt}-\partial_x\!\big[(c_\theta^2+\alpha\,\phi^2)\,\theta_x\big]=0.\]
Local dispersion for small perturbations on a slowly varying background:
\[\omega^2(x)=c_\phi^2 k^2 + V''(\phi(x)) + \alpha\,\theta_x^2(x).\]
Spatial variation of $V''(\phi)$ and $\theta_x$ **biases the group velocity** $v_g=\partial\omega/\partial k$, producing net drift.

## 1) Imports & Parameters
- Dimensionless units
- Exponential potential $V(\phi)=\rho_0\,e^{-\phi/\phi_s}$
- Energy conservation guard (adaptive `dt` if drift exceeds tolerance)

In [ ]:
import numpy as np, matplotlib.pyplot as plt

# ---------- Parameters (dimensionless PQF) ----------
c_phi = 1.0
c_th  = 1.0
rho0  = 1.0
phi_s = 1.0
alpha = 0.6

# Potential & derivatives
def V(phi):   return rho0*np.exp(-phi/phi_s)
def Vp(phi):  return -(rho0/phi_s)*np.exp(-phi/phi_s)
def Vpp(phi): return (rho0/(phi_s**2))*np.exp(-phi/phi_s)

# Grid
N  = 4096
L  = 400.0
dx = L/N
x  = np.linspace(-L/2, L/2, N, endpoint=False)

# Operators (periodic)
def dxx(f): return (np.roll(f,-1)-2*f+np.roll(f,1))/dx**2
def dx1(f): return (np.roll(f,-1)-np.roll(f,1))/(2*dx)

# Time step (CFL)
CFL = 0.4
dt  = CFL * dx / max(c_phi, c_th)

# Simulation length
steps      = 12000
out_every  = 200

# Energy guard
ENERGY_TOL = 1e-4     # relative drift tolerance (0.01%)
ADAPT_RATE = 0.8      # dt multiplier if violation
MAX_ADAPT  = 5        # stop after this many adaptations

## 2) Initial Conditions
- Localized carrier+envelope in $\phi$
- Gentle phase slope in $\theta$ → $\,\theta_x \neq 0$

In [ ]:
# Initial fields
phi0   = 0.5
k0     = 0.25
env    = np.exp(-(x+40.0)**2/200.0)
phi    = phi0 + 0.08*env*np.cos(k0*x)
phi_t  = np.zeros_like(phi)

theta   = 0.02*np.tanh(x/30.0)  # background phase slope
theta_t = np.zeros_like(theta)

# Accelerations (EoMs)
def acc_phi(phi, theta):
    thx = dx1(theta)
    return c_phi**2 * dxx(phi) - Vp(phi) - alpha*phi*(thx**2)

def acc_th(phi, theta):
    thx  = dx1(theta)
    term = (c_th**2 + alpha*phi**2)*thx
    return (np.roll(term,-1) - np.roll(term,1))/(2*dx)

# Leapfrog bootstrap (half step)
phi_t   += 0.5*dt*acc_phi(phi, theta)
theta_t += 0.5*dt*acc_th(phi, theta)

## 3) Energy, Diagnostics & Drift Measure
We monitor the Hamiltonian density
$$\mathcal{H}=\tfrac12\phi_t^2+\tfrac{c_\phi^2}{2}\phi_x^2+\tfrac12\theta_t^2+\tfrac{c_\theta^2}{2}\theta_x^2+V(\phi)+\tfrac{\alpha}{2}\phi^2\theta_x^2,$$
track relative drift $\Delta H/H_0$, and compute a robust **envelope center** $x_c(t)$.

In [ ]:
def energy_density(phi, phi_t, theta, theta_t):
    phx = dx1(phi)
    thx = dx1(theta)
    return (0.5*phi_t**2 + 0.5*(c_phi**2)*phx**2 +
            0.5*theta_t**2 + 0.5*(c_th**2)*thx**2 +
            V(phi) + 0.5*alpha*phi**2*thx**2)

def envelope_center(f):
    w = f - f.min()
    s = w.sum()
    if s <= 0: return 0.0
    w /= s
    return np.sum(w*x)

H0 = energy_density(phi, phi_t, theta, theta_t).mean()
centers, times, relH = [], [], []
adapt_count = 0

## 4) Main Integrator with Energy Guard
If $|\Delta H/H_0|$ > tolerance, we **adapt dt** (reduce by 20%) up to a cap, then stop to avoid misleading results.

In [ ]:
for n in range(steps):
    # Advance momenta
    phi_t   += dt * acc_phi(phi, theta)
    theta_t += dt * acc_th(phi, theta)
    # Advance fields
    phi     += dt * phi_t
    theta   += dt * theta_t

    if n % out_every == 0:
        H = energy_density(phi, phi_t, theta, theta_t).mean()
        rel = (H - H0)/abs(H0) if H0 != 0 else 0.0
        relH.append(rel); times.append(n*dt); centers.append(envelope_center(phi))
        print(f"step {n:6d} | ΔH/H0={rel: .3e} | center≈{centers[-1]:.2f} | dt={dt:.3e}")
        # Energy guard
        if abs(rel) > ENERGY_TOL:
            if adapt_count < MAX_ADAPT:
                dt *= ADAPT_RATE
                adapt_count += 1
                print(f"  ⚠ Energy drift exceeded {ENERGY_TOL:.1e}. Reducing dt → {dt:.3e} (#{adapt_count})")
            else:
                print("  ❌ Energy drift persists after adaptations. Stopping to avoid artifacts.")
                break

## 5) Results
- **Drift**: plot envelope center $x_c(t)$
- **Energy check**: relative drift over time
- **Final profile**: inspect $\phi(x)$ at the end

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(10,3.5))

ax[0].plot(times, centers, '-')
ax[0].set_xlabel('time'); ax[0].set_ylabel('envelope center  $x_c(t)$')
ax[0].set_title('Derived GW-like envelope drift')
ax[0].grid(alpha=0.3)

ax[1].plot(times, relH, '-')
ax[1].axhline(0, color='k', lw=1)
ax[1].set_xlabel('time'); ax[1].set_ylabel('relative energy drift  ΔH/H₀')
ax[1].set_title('Energy conservation check')
ax[1].grid(alpha=0.3)

plt.tight_layout(); plt.show()

plt.figure(figsize=(7,3))
plt.plot(x, phi)
plt.xlabel('x'); plt.ylabel('φ(x)  (final)')
plt.title('Final φ profile')
plt.grid(alpha=0.3); plt.tight_layout(); plt.show()

## 6) Notes for Reviewers
- **No ad-hoc terms**: every term in the EoMs comes from the LPQF Lagrangian.
- **Mechanism**: drift arises from spatial variation in $V''(\phi)$ and $\theta_x$, biasing $v_g$.
- **Quality control**: energy guard with adaptive `dt`; stop on persistent drift.
- **Reproducibility**: 1D periodic box; change `N, L, dt` to probe convergence.
- **Next**: add 2D ray-bending (graded-index) emergent lensing as a separate notebook.